

![](https://i.imgur.com/La3jjO4.png)

**© VTCA-COTAI 2020 - Internal use only!**

## <a id='mab'>Phân tích bài toán Multi-Armed Bandit với Showing advertisement.</a>
Ví dụ, tưởng tượng rằng ta đang thực hiện chiến dịch marketing cho trang web với hai lựa chọn chạy quảng cáo. Giả sử mục tiêu là hiển thị quảng cáo có tỉ lệ nhấp chuột CTR (click through rate) cao nhất để thu hút lượng truy cập cao nhất có thể. Nhưng ta không có bất kỳ thông tin trước đó để biết là nên hiển thị mấy cái đó như thế nào. Cách điển hình là sẽ thử nghiệm chiếu cả hai như nhau, tính toán CTR của từng cái, sau đó, ở một vài thời điểm sẽ chuyển sang quảng cáo có CTR cao hơn. Ta sẽ chiếu cả hai quảng cáo trong bao lâu trước khi tập trung chiếu ổn định một quảng cáo trội hơn (để thu lợi nhiều hơn)? Trong những trường hợp này, có vẻ như đoán mò có lẽ là cách đánh cược tốt nhất. Thực tế thì đó là cách thường dùng. 

Phương pháp ta dùng để giải bài toán này gọi là `Tướng cướp nhiều tay` (multi armed bandit - MAB). (Và thường người ta cũng gọi bài toán trên là bài toán MAB)

Đọc thêm [thuật toán MAB](https://en.wikipedia.org/wiki/Multi-armed_bandit) để hiểu nó là gì và hoạt động thế nào. Cơ bản thì, thuật toán MAB là một phương pháp cận tối ưu (near-optimal) để giải quyết việc đánh đổi giữa khai phá (explore) hay khai thác (exploit) khi mà ta không biết liệu nên khai phá các lựa chọn khả dĩ để tìm ra lựa chọn tốt nhất hay tập trung khai thác một lựa chọn mà ta cảm giác rằng nó tốt nhất từ những lần khai phá hữu hạn mà ta đã làm trước đó. 

Bây giờ ta sẽ tìm hiểu các thuật toán khác nhau để giải quyết bài toán MAB. Chúng đều có những cách tiếp cận và tỷ lệ giữa khai phá và khai thác khác nhau.

**CTR** là tỷ lệ giữa số lần click chuột với số lần hiển thị của một quảng cáo. Ví dụ, nếu một quảng cáo được chiếu 100 lần và được click xem 10 lần thì CTR = 10/100 = 0.1.

**Regret** là sự khác nhau giữa CTR khả dĩ cao nhất và CTR tính được. Ví dụ, nếu quảng cáo A có CTR đã biết là 0.1 và quảng cáo B có CTR đã biết là 0.3, mỗi khi ta chiếu quảng cáo A, ta có regret bằng 0.3 - 0.1 = 0.2. Đây có vẻ là một sự khác nhau nho nhỏ cho đến khi ta xét đến một quảng cáo có thể được hiển thị 1 triệu lần chỉ trong vài giờ.

Tiếp theo, ta sẽ thực thi để tìm ra thuật toán nào sẽ tốt nhất trong việc tối thiểu hóa regret.
Bốn cách thực thi mà ta sẽ dùng gồm:

1. Chọn Ngẫu Nhiên (Random Selection)
2. Epsilon Greedy
3. Lấy mẫu Thompson
4. Cận trên của khoảng tin cậy (Upper Confidence Bound - UCB1)

Sau đây ta sẽ mô phỏng lại 4 cách trên.

Giả sử rằng ta biết trước CTR. Như vậy, ta có thể mô phỏng một cú click (hoặc không) của một quảng cáo cho trước. Ví dụ, nếu ta chiếu quảng cáo A, với CTR đã biết là 28%, ta có thể giả sử rằng quảng cáo sẽ được click chuột trong 28% thời gian hiển thị và đưa nó vào mô phỏng bên dưới.


In [25]:
import tensorflow

ImportError: Traceback (most recent call last):
  File "C:\Users\shopeevn\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\shopeevn\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\shopeevn\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\shopeevn\anaconda3\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\shopeevn\anaconda3\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: The specified module could not be found.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/inst

##Import thư viện

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import beta, bernoulli
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import subplots
init_notebook_mode(connected=True)
import random
import math
import plotly
plotly.io.renderers.default = 'colab'
np.random.seed(123)

In [2]:
def algorithm_performance():
    """
    Function that will show the performance of each algorithm we will be using in this tutorial.
    Hàm biểu diễn việc thực hiện mỗi thuật toán ta sẽ dùng trong phần hướng dẫn này.
    """
    
    ## calculate how many time each Ad has been choosen
    ## tính số lần mỗi quảng cáo được chọn
    print('index_list',index_list)
    count_series = pd.Series(index_list).value_counts(normalize=True)
    print('Ad #0 has been shown', count_series[0]*100, '% of the time.')
    print('Ad #1 has been shown', count_series[1]*100, '% of the time.')
    
    print('Total Reward (Number of Clicks):', total_reward) ## print total Reward
    
    x = np.arange (0, n, 1)

    # plot the calculated CTR for Ad #0
    # tính và vẽ đồ thị CTR đã tính được cho quảng cáo #0
    data1 = go.Scatter(x=x,
                       y=ctr[0],
                       name='Calculated CTR #0',
                       line=dict(color=('rgba(10, 108, 94, .7)'),
                                 width=2))

    ## plot the line with actual CTR for Ad #0
    ## vẽ đường thẳng biểu diễn giá trị CTR thực tế của quáng cáo #0
    data2 = go.Scatter(x=[0, n],
                       y=[ACTUAL_CTR[0]] * 2,
                       name='Actual CTR #0 value',
                       line = dict(color = ('rgb(205, 12, 24)'),
                                   width = 1,
                                   dash = 'dash'))

    ## plot the calculated CTR for Ad #1
    ## tính và vẽ đồ thị CTR đã tính được cho quảng cáo #1
    data3 = go.Scatter(x=x,
                       y=ctr[1],
                       name='Calculated CTR #1',
                       line=dict(color=('rgba(187, 121, 24, .7)'),
                                 width=2))

    ## plot the line with actual CTR for Ad #1
    ## vẽ đường thẳng biểu diễn giá trị CTR thực tế của quáng cáo #1
    data4 = go.Scatter(x=[0, n],
                       y=[ACTUAL_CTR[1]] * 2,
                       name='Actual CTR #1 value',
                       line = dict(color = ('rgb(205, 12, 24)'),
                                   width = 1,
                                   dash = 'dash'))

    ## plot the Regret values as a function of trial number
    ## biểu diễn giá trị Regret: là một hàm có tham số là số lần thử nghiệm
    data5 = go.Scatter(x=x,
                       y=regret_list,
                       name='Regret')

    layout = go.Layout(title='Simulated CTR Values and Algorithm Regret',
                       xaxis={'title': 'Trial Number'},
                       yaxis1={'title': 'CTR value'},
                       yaxis2={'title': 'Regret Value'}
                       )
    fig = subplots.make_subplots(rows=2, cols=1, print_grid=False, shared_yaxes=True, shared_xaxes=True)

    fig.append_trace(data1, 1, 1)
    fig.append_trace(data2, 1, 1)
    fig.append_trace(data3, 1, 1)
    fig.append_trace(data4, 1, 1)
    fig.append_trace(data5, 2, 1)

    fig['layout'].update(layout)
    iplot(fig, show_link=False)

In [3]:
# Để mô phỏng ta giả sử biết giá trị CTR
ACTUAL_CTR = [.45, .65]
print('Actual CTR for Ad #0 is:', ACTUAL_CTR[0])
print('Actual CTR for Ad #1 is:', ACTUAL_CTR[1])

Actual CTR for Ad #0 is: 0.45
Actual CTR for Ad #1 is: 0.65


# <a id='random'>Chọn Ngẫu Nhiên (Random Selection)</a> 

Thuật toán Chọn ngẫu nhiên không làm chuyện khai phá, nó chỉ chọn ngẫu nhiên quảng cáo để hiển thị.

Bạn có thể hình dung như việc tung đồng xu vậy - nếu mặt ngửa ta chiếu quảng cáo #0, nếu mặt sấp ta chiếu quảng cáo #1. Vì vậy nếu ta có 2 quảng cáo, mỗi quảng cáo sẽ xuất hiện ~50% (=100%/2) số lần trình chiếu. Đến đây có thể bạn đã đoán được hạn chế của giải thuật này là gì rồi, nhưng hãy thử chạy mô phỏng xem thế nào.

## Set the initial values

In [4]:
## For each alrorithm we will perform 1000 trials
n = 100
regret = 0 
total_reward = 0
regret_list = [] ## list for collecting the regret values for each impression (trial)
ctr = {0: [], 1: []} ## lists for collecting the calculated CTR 
index_list = [] ## list for collecting the number of randomly choosen Ad

## set the initial values for impressions and clicks 
impressions = [0,0] 
clicks = [0,0]

In [5]:
for i in range(n):    
    random_index = np.random.randint(0,2,1)[0] ## randomly choose the value between [0,1]
                                               ## chọn ngẫu nhiên giá trị nguyên trong khoảng [0,1]
    index_list.append(random_index) ## add the value to list
                                    ## thêm giá trị cho danh sách
    # print('danh sách:',index_list)
    impressions[random_index] += 1 ## add 1 impression value for the choosen Ad
                                   ## tăng 1 lượt chiếu cho quảng cáo được chọn
    # print('đếm lượt chiếu cho quảng cáo:', impressions)
    did_click = bernoulli.rvs(ACTUAL_CTR[random_index]) ## simulate if the person clicked on the ad usind Actual CTR value
    # print('khách hàng đã click vào quảng cáo:',did_click)
    if did_click:
        clicks[random_index] += did_click ## if person clicked add 1 click value for the choosen Ad
    ## calculate the CTR values and add them to list
    ## tính giá trị CTR và thêm nó vào danh sách
    if impressions[0] == 0:
        ctr_0 = 0
    else:
        ctr_0 = clicks[0]/impressions[0]
    if impressions[1] == 0:
        ctr_1 = 0
    else:
        ctr_1 = clicks[1]/impressions[1]
    ctr[0].append(ctr_0)
    ctr[1].append(ctr_1)
    # print('danh sách ghi lại giá trị CTR_0 tính toán được:', ctr[0])
    # print('danh sách ghi lại giá trị CTR_1 tính toán được', ctr[1])

    ## calculate the regret and reward
    ## tính toán regret và reward
    # print('ACTUAL_CTR',ACTUAL_CTR)
    # print('ACTUAL_CTR[random_index]',ACTUAL_CTR[random_index])
    regret += max(ACTUAL_CTR) - ACTUAL_CTR[random_index]
    # print('regret',regret)
    regret_list.append(regret)
    # print("regret list",regret_list)
    total_reward += did_click
    # print("total_reward",total_reward)



## performance of algorithm

In [6]:
algorithm_performance()

index_list [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0]
Ad #0 has been shown 49.0 % of the time.
Ad #1 has been shown 51.0 % of the time.
Total Reward (Number of Clicks): 56


Cả hai quảng cáo đều được chiếu với số lần bằng nhau và qua càng nhiều lượt thử nghiệm thì giá trị CTR tính toán được sẽ càng gần với giá trị mà ta đã biết trước. Tuy nhiên, regret vẫn liên tục tăng do thuật toán không học được gì và không cập nhật gì so với thông tin thu được. Regret đang tăng lên này chính xác là những gì ta hy vọng sẽ giảm thiểu được bằng các phương pháp "thông minh hơn".

## Save result

In [7]:
## save the reward and regret values for future comparison
random_dict = {'reward':total_reward,
               'regret_list':regret_list, 
               'ads_count':pd.Series(index_list).value_counts(normalize=True)}
print(random_dict)

{'reward': 56, 'regret_list': [0.2, 0.4, 0.6000000000000001, 0.8, 1.0, 1.0, 1.0, 1.2, 1.2, 1.4, 1.5999999999999999, 1.5999999999999999, 1.5999999999999999, 1.5999999999999999, 1.7999999999999998, 1.7999999999999998, 1.9999999999999998, 1.9999999999999998, 1.9999999999999998, 2.1999999999999997, 2.4, 2.4, 2.6, 2.8000000000000003, 2.8000000000000003, 3.0000000000000004, 3.2000000000000006, 3.400000000000001, 3.600000000000001, 3.600000000000001, 3.600000000000001, 3.800000000000001, 3.800000000000001, 4.000000000000001, 4.200000000000001, 4.200000000000001, 4.200000000000001, 4.200000000000001, 4.200000000000001, 4.200000000000001, 4.200000000000001, 4.200000000000001, 4.400000000000001, 4.400000000000001, 4.600000000000001, 4.800000000000002, 4.800000000000002, 4.800000000000002, 4.800000000000002, 4.800000000000002, 5.000000000000002, 5.000000000000002, 5.000000000000002, 5.000000000000002, 5.200000000000002, 5.200000000000002, 5.400000000000002, 5.600000000000002, 5.8000000000000025, 

# <a id='epsilon'>Epsilon Greedy</a> 

Cách tiếp cận tiếp theo là thuật toán Epsilon-Greedy. Logic như sau:

1. Chạy thử nghiệm từng quảng cáo vài lần đầu (**Khai phá**).

2. Trong giai đoạn khai phá ban đầu này, chọn ra quảng cáo có số điểm cao nhất.
3. Thiết lập giá trị Epsilon,  **$\epsilon$**.

4. Chạy thử nghiệm với việc chọn biến thể (variant) trội hơn với **$(1-\epsilon)$** (%)thời gian và các lựa chọn khác với **$\epsilon\$** (%)thời gian (**Khai thác**).

Sau đây là cách thuật toán thực thi:


## Set initial values

In [8]:
e = .05 ## set the Epsilon value
n_init = 10 ## number of impressions to choose the winning Ad
             ## số lần hiển thị để chọn quảng cáo trội hơn
## set the initial values for impressions and clicks 
impressions = [0,0] 
clicks = [0,0] 

In [9]:
for i in range(n_init):
    ## chọn ra ngẫu nhiên 1 số nguyên có giá trị từ [0,2)
    random_index = np.random.randint(0,2,1)[0]
    # print('quảng cáo được chiếu',random_index)
    impressions[random_index] += 1
    # print('số lần chiếu của 2 loại quảng cáo',impressions)
    did_click = bernoulli.rvs(ACTUAL_CTR[random_index])
    if did_click:
        clicks[random_index] += did_click
        # print('số lần khách hàng click chuột vào xem quảng cáo',clicks)
ctr_0 = clicks[0] / impressions[0]
ctr_1 = clicks[1] / impressions[1]
win_index = np.argmax([ctr_0, ctr_1]) ## select the Ad number with the highest CTR
                                      ## chọn quảng cáo có CTR cao nhất

# print('After', n_init, 'initial trials Ad #', win_index, 'got the highest CTR =', round(np.max([ctr_0, ctr_1]), 2), 
      # '(Real CTR value is', ACTUAL_CTR[win_index], ').'
      # '\nIt will be shown', (1-e)*100, '% of the time.')
# print(ctr_0, ctr_1)

## Set initial values

In [10]:
regret = 0 
total_reward = 0
regret_list = [] 
ctr = {0: [], 1: []}
index_list = [] 
impressions = [0,0] 
clicks = [0,0]

In [11]:


for i in range(n):    
    # print('win_index', win_index)
    epsilon_index = random.choices([win_index, 1-win_index], [1-e, e])[0]
    index_list.append(epsilon_index)
    # print('epsilon_index',epsilon_index)
    # print('index_list',index_list)
    impressions[epsilon_index] += 1
    # print('impressions',impressions)
    did_click = bernoulli.rvs(ACTUAL_CTR[epsilon_index])
    if did_click:
        clicks[epsilon_index] += did_click
    
    if impressions[0] == 0:
        ctr_0 = 0
    else:
        ctr_0 = clicks[0]/impressions[0]
        
    if impressions[1] == 0:
        ctr_1 = 0
    else:
        ctr_1 = clicks[1]/impressions[1]
        
    ctr[0].append(ctr_0)
    ctr[1].append(ctr_1)
    # print('ctr',ctr)
    
    # print('ACTUAL_CTR',ACTUAL_CTR)
    # print('ACTUAL_CTR[epsilon_index]',ACTUAL_CTR[epsilon_index])

    regret += max(ACTUAL_CTR) - ACTUAL_CTR[epsilon_index]
    regret_list.append(regret)
    total_reward += did_click
    # print("regret", regret)
    # print('regret_list',regret_list)
    # print('total_reward',total_reward)

In [12]:
algorithm_performance()

index_list [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Ad #0 has been shown 7.000000000000001 % of the time.
Ad #1 has been shown 93.0 % of the time.
Total Reward (Number of Clicks): 65


Lưu ý, nhìn vào đồ thị regret, ta nhận ra regret đã giảm thế nào! Thuật toán Epsilon-Greedy có vẻ thực thi tốt hơn Chọn ngẫu nhiên. Nhưng bạn có thể thấy được vấn đề ở đây là quảng cáo trội hơn ở giai đoạn khai phá sẽ không hẳn là quảng cáo tối ưu, và thực tế thì ta có thể khai thác quảng cáo gần tối ưu. Điều này làm tăng regret và giảm reward. Theo **Luật số lớn (Law of large numbers - LLN)** thì ta càng làm nhiều lượt thử nghiệm ban đầu, ta càng có thể chọn được quảng cáo trội hơn chính xác. Nhưng trong Marketing không phải lúc nào ta cũng muốn dựa vào cơ hội và hy vọng chạm được 'số lần thử nghiệm lớn' kia. 

> - Theo lý thuyết thống kê, **Luật số lớn** là một định lý mô tả kết quả của việc thực thi cùng một thử nghiệm với số lần lớn. Và theo đó, trung bình của các kết quả thu được từ số lượng lớn các thử nghiệm  xu hướng gần với giá trị kỳ vọng, và sẽ càng gần hơn khi ta thực thi nhiều lượt thử nghiệm hơn nữa. 

- Ưu điểm: Ta có thể kiểm soát tần suất hiển thị quảng cáo trội hơn sau các lượt thử nghiệm ban đầu bằng cách chọn các giá trị $\epsilon$ khác nhau.

- Khuyết điểm: Ta không thể biết được ϵ tối ưu nhất cho bài toán của bạn. Nếu ϵ của bạn quá lớn, nghĩa là bạn chọn số lần thử dành cho mục đích khai thác quá nhiều, nhưng nếu số ϵ của bạn quá nhỏ, bạn sẽ có khả năng đang khai thác không hiệu quả.

##Save result

In [13]:
epsilon_dict = {'reward':total_reward, 
                'regret_list':regret_list, 
                'ads_count':pd.Series(index_list).value_counts(normalize=True)}
print(epsilon_dict)

{'reward': 65, 'regret_list': [0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4], 'ads_count': 1    0.93
0    0.07
dtype: float64}


# <a id='ts'>Lấy mẫu Thompson</a> 


Phần khai phá của Lấy mẫu Thompson tinh vi hơn thuật toán $\epsilon$-Greedy. Ta xài **phân phối Beta (Beta distribution)**\* ở đây, tuy nhiên Lấy mẫu Thompson có thể được khái quát hóa để lấy mẫu từ bất kỳ phân phối nào trên các tham số.

> Trong thống kê và lý thuyết xác suất, **phân phối Beta** là một họ của các phân phối xác suất liên tục xác định trên khoảng [0,1] được tham số hóa bởi hai tham số dương, ký hiệu là $\alpha$ và $\beta$, xuất hiện dưới dạng luỹ thừa của biến ngẫu nhiên và kiểm soát hình dạng của phân phối.

>Nếu muốn tìm hiểu nhiều hơn về phân phối Beta thì [đây](http://varianceexplained.org/statistics/beta_distribution_and_baseball/) là bài viết cực kỳ hữu ích.

Logic như sau:


1. Chọn các phân phối tiền nghiệm (prior distribution) cho tham số $\alpha$ và $\beta$.
2. Tính giá trị $\alpha$ và $\beta$: $\alpha = \text{trước + số lượt click (prior + hits)}$, $\beta = \text{trước + số bỏ lỡ (prior + misses)}$. Trong trường hợp này, số lượt click là hits - number of click, số lần hiển thị không có lượt click là misses - number of impression. Phân phối tiền nghiệm (priors) sẽ hữu dụng nếu ta đã có vài thông tin trước về CTR thực tế của các quảng cáo. Nhưng ở đây ta không có, vì vậy ta sẽ dùng 1.0.
3. Ước tính CTR thực tế bằng cách lấy mẫu các giá trị của phân phối Beta cho mỗi biến thể $B(\alpha_i, \beta_i)$ và chọn mẫu với giá trị cao nhất (là CTR ước tính được).
4. Lặp lại bước 2-3.

##Set initial values

In [14]:
n = 100
regret = 0 
total_reward = 0
regret_list = [] 
ctr = {0: [], 1: []}
index_list = [] 
impressions = [0,0] 
clicks = [0,0]
# alpha, beta ban đầu được khởi tạo bằng 1.
priors = (1, 1)
win_index = np.random.randint(0,2,1)[0] ## randomly choose the first shown Ad



In [15]:
for i in range(n):    
    
    impressions[win_index] += 1
    did_click = bernoulli.rvs(ACTUAL_CTR[win_index])
    if did_click:
        clicks[win_index] += did_click
    # print("priors[0]",priors[0])
    # print("clicks[0]",clicks[0])
    # print("impressions[0]",impressions[0])
    # print()
    # print("priors[1]",priors[1])
    # print("clicks[1]",clicks[1])
    # print("impressions[1]",impressions[1])
    # print(priors[0]+clicks[0], priors[1] + impressions[0] - clicks[0])
    ctr_0 = random.betavariate(priors[0]+clicks[0], priors[1] + impressions[0] - clicks[0])
    # print("ctr_0",ctr_0)
    # print()
    # print(priors[0]+clicks[1], priors[1] + impressions[1] - clicks[1])
    ctr_1 = random.betavariate(priors[0]+clicks[1], priors[1] + impressions[1] - clicks[1])
    # print("ctr_1",ctr_1)
    win_index = np.argmax([ctr_0, ctr_1])
    # print("win_index", win_index)
    index_list.append(win_index)
    # print("index_list", index_list)
    ctr[0].append(ctr_0)
    # print("ctr_quang_cao_0",ctr[0])
    ctr[1].append(ctr_1)
    # print("ctr_quang_cao_1",ctr[1])
    regret += max(ACTUAL_CTR) - ACTUAL_CTR[win_index]
    regret_list.append(regret)    
    total_reward += did_click

In [16]:
## plot the Beta distributions
## vẽ đồ thị phân phối Beta
x = np.arange (0, 1, 0.01)
y = beta.pdf(x, priors[0]+clicks[0], priors[1] + impressions[0] - clicks[0])
y /= y.max() ## normalize
             ## hiệu chỉnh

data1 = go.Scatter(x=x,
                   y=y,
                   name='Beta Distribution (Ad #0)',
                   marker = dict(color=('rgba(10, 108, 94, 1)')),
                   fill='tozeroy',
                   fillcolor = 'rgba(10, 108, 94, .7)')

data2 = go.Scatter(x = [ACTUAL_CTR[0]] * 2,
                   y = [0, 1],
                   name = 'Actual CTR #0 Value',
                   mode='lines',
                   line = dict(
                       color = ('rgb(205, 12, 24)'),
                       width = 2,
                       dash = 'dash'))

y = beta.pdf(x, priors[0]+clicks[1], priors[1] + impressions[1] - clicks[1])
y /= y.max()

data3 = go.Scatter(x=x,
                   y=y,
                   name='Beta Distribution (Ad #1)',
                   marker = dict(color=('rgba(187, 121, 24, 1)')),
                   fill='tozeroy',
                   fillcolor = 'rgba(187, 121, 24, .7)')

data4 = go.Scatter(x = [ACTUAL_CTR[1]] * 2,
                   y = [0, 1],
                   name = 'Actual CTR #1 Value',
                   mode='lines',
                   line = dict(
                       color = ('rgb(205, 12, 24)'),
                       width = 2,
                       dash = 'dash'))

layout = go.Layout(title='Beta Distributions for both Ads',
                   xaxis={'title': 'Possible CTR values'},
                   yaxis={'title': 'Probability Density'})

fig = go.Figure(data=[data1, data2, data3, data4], layout=layout)

# fig = subplots.make_subplots(rows=1, cols=2, print_grid=False, shared_xaxes=False,
#                           subplot_titles=('Beta Distribution (Ad #0)','Beta Distribution (Ad #1)'))

# fig.append_trace(data1, 1, 1)
# fig.append_trace(data2, 1, 1)
# fig.append_trace(data3, 1, 2)
# fig.append_trace(data4, 1, 2)

# fig['layout'].update(showlegend=False)

iplot(fig, show_link=False)

In [17]:
algorithm_performance()

index_list [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Ad #0 has been shown 37.0 % of the time.
Ad #1 has been shown 63.0 % of the time.
Total Reward (Number of Clicks): 58


Regret là thấp nhất mà ta từng thấy đến giờ. Mỗi lần tăng regret xảy ra khi quảng cáo #0 được chọn. Trên đồ thị CTR, ta có thể thấy rằng khi bắt đầu, các giá trị có màu xanh (giá trị CTR được lấy mẫu Thompson cho quảng cáo #0) thường cao hơn giá trị tan (giá trị CTR được lấy mẫu Thompson cho quảng cáo #1), dẫn đến quảng cáo #0 được chiếu.

Lưu ý về sự khác nhau trong các thay đổi cho mỗi quảng cáo. Thuật toán khai phá liên tục, sau đó khai thác một cách tự nhiên các biến thể quảng cáo với mẫu cao nhất được lấy từ phân phối Beta xấp xỉ. Điều này có thể được biểu diễn ở đồ thị trên cùng của các phân phối. Phân phối Beta cho quảng cáo #1 thì cao hơn nhiều và hẹp hơn nhiều so với quảng cáo #0, đây chính xác là điều ta muốn!

##Save result

In [18]:
thompson_dict = {'reward':total_reward, 
                 'regret_list':regret_list, 
                 'ads_count'  :pd.Series(index_list).value_counts(normalize=True)}

# <a id='ucb'>Cận trên của khoảng tin cậy (Upper Confidence Bound - UCB)</a> 


Không giống như thuật toán Lấy mẫu Thompson, UCB quan tâm nhiều hơn về tính không chắc chắn (độ lệch lớn) của mỗi biến thể. Càng không chắc chắn về một quảng cáo, thì càng phải ưu tiên tập trung khai phá.

Thuật toán chọn ra biến thể với giá trị cận trên của khoảng tin cậy cao nhất (UCB) - cũng có nghĩa là dự đoán biến thể cho reward cao nhất. Được định nghĩa như sau:

$UCB = \bar x_i + \sqrt{\frac{2 \cdot \log{t}}{n}}$ ,

trong đó:
- $\bar x_i$ - tỷ lệ CTR tại bước thứ $i$,

- $t$ - tổng số lượt chiếu của tất cả quảng cáo,

- $n$ - tổng số lượt chiếu của quảng cáo được chọn


Logic khá là dễ hiểu:

1. Tính UCB của tất cả biến thể.
2. Chọn biến thể với UCB cao nhất.
3. Thực hiện bước 1.

In [19]:
n =100
regret = 0 
total_reward = 0
regret_list = [] 
index_list = [] 
impressions = [0,0] 
clicks = [0,0]
ctr = {0: [], 1: []}
total_reward = 0

for i in range(n):
    index = 0
    max_upper_bound = 0
    for k in [0,1]:
        # print("impressions[k]",impressions[k])
        if (impressions[k] > 0):
            CTR = clicks[k] / impressions[k]
            delta = math.sqrt(2 * math.log(i+1) / impressions[k])
            upper_bound = CTR + delta
            # print(upper_bound)
            ctr[k].append(CTR)
        else:
            upper_bound = 1e400
        if upper_bound > max_upper_bound:
            # print(max_upper_bound)
            max_upper_bound = upper_bound
            index = k
            # print(index)
    index_list.append(index)
    impressions[index] += 1
    reward = bernoulli.rvs(ACTUAL_CTR[index])
    
    clicks[index] += reward
    total_reward += reward
    
    regret += max(ACTUAL_CTR) - ACTUAL_CTR[index]
    regret_list.append(regret)

In [20]:
algorithm_performance()

index_list [0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]
Ad #0 has been shown 24.0 % of the time.
Ad #1 has been shown 76.0 % of the time.
Total Reward (Number of Clicks): 60


Ta có thể thấy rằng regret tăng khi thuật toán cố gắng giảm độ không chắc chắc của CTR của quảng cáo #0 bằng cách chọn chiếu nó nhiều hơn.

Nó có thể hữu dụng khi ta muốn mô hình chọn được quảng cáo tốt nhất thường xuyên hơn, nhưng vẫn quan tâm đến việc giảm sự độ không chắc chắn của cả hai quảng cáo.

## Save result

In [21]:
ucb1_dict = {'reward':total_reward, 
             'regret_list':regret_list, 
             'ads_count':pd.Series(index_list).value_counts(normalize=True)}

# <a id='comparison'>So sánh và Kết luận</a> 
Đầu tiên, rõ ràng rằng ta muốn chiếu quảng cáo #1 thường xuyên hơn vì CTR thực tế của nó là 0.65. Hãy cùng nhìn xem tỷ lệ số lần quảng cáo đúng được chọn ở mỗi thuật toán.

In [22]:
data1 = go.Bar(x=['Random Selection', 'Epsilon Greedy', 'Thompson Sampling', 'UCB1'],
               y=[random_dict['ads_count'][0], 
                  epsilon_dict['ads_count'][0], 
                  thompson_dict['ads_count'][0],
                  ucb1_dict['ads_count'][0]],
               name='Ad #0',
               marker=dict(color='rgba(10, 108, 94, .7)'))

data2 = go.Bar(x=['Random Selection', 'Epsilon Greedy', 'Thompson Sampling', 'UCB1'],
               y=[random_dict['ads_count'][1], 
                  epsilon_dict['ads_count'][1], 
                  thompson_dict['ads_count'][1],
                  ucb1_dict['ads_count'][1]],
               name='Ad #1',
               marker=dict(color='rgba(187, 121, 24, .7)'))

data = [data1, data2]
layout = go.Layout(title='Ratio of appearance of both Ads throughout the trials',
                   xaxis={'title': 'Algorithm'},
                   yaxis={'title': 'Ratio'},
                   barmode='stack')

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Thuật toán trội hơn là Lấy mẫu Thompson và Epsilon-Greedy vì chúng chiếu đúng quảng cáo #1 hầu hết thời gian.

In [23]:
data1 = go.Scatter(
    x=np.arange (0, n, 1),
    y=random_dict['regret_list'],
    name='Random Selection',
    marker=dict(color='#ffcc66')
)
data2 = go.Scatter(
    x=np.arange (0, n, 1),
    y=epsilon_dict['regret_list'],
    name='e-Greedy',
    marker=dict(color='#0099ff')
)
data3 = go.Scatter(
    x=np.arange (0, n, 1),
    y=thompson_dict['regret_list'],
    name='Thompson Sampling',
    marker=dict(color='#ff3300')
)
data4 = go.Scatter(
    x=np.arange (0, n, 1),
    y=ucb1_dict['regret_list'],
    name='UCB1',
    marker=dict(color='#33cc33')
)

layout = go.Layout(
    title='Regret by the Algorithm',
    xaxis={'title': 'Trial'},
    yaxis={'title': 'Regret'}
)

data = [data1, data2, data3, data4]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

Khi nói đến thuật toán Lấy mẫu Thompson và Epsilon-Greedy đã chọn quảng cáo với CTR cao hơn (#1) trong hầu hết lượt chiết, không ngạc nhiên vì nó cho giá trị regret của chúng là thấp nhất.

In [24]:
data = go.Bar(
    x=[ucb1_dict['reward'], thompson_dict['reward'], epsilon_dict['reward'], random_dict['reward']],
    y=['UCB1', 'Thompson Sampling', 'e-Greedy','Random Selection'],
    orientation = 'h',
    marker=dict(color=['#33cc33', '#ff3300', '#0099ff', '#ffcc66']),
    opacity=0.7
)

text = go.Scatter(
    x=[ucb1_dict['reward'], thompson_dict['reward'], epsilon_dict['reward'], random_dict['reward']],
    y=['UCB1', 'Thompson Sampling', 'e-Greedy', 'Random Selection'],
    mode='text',
    text=[ucb1_dict['reward'], thompson_dict['reward'], epsilon_dict['reward'], random_dict['reward']],
    textposition='middle left',
    line = dict(
        color = ('rgba(255,141,41,0.6)'),
        width = 1
    ),
    textfont=dict(
        family='sans serif',
        size=16,
        color='#000000'
    )
)

data = [data,text]

layout = go.Layout(
    title='Total Reward by Algorithms',
    xaxis={'title': 'Total Reward (Clicks)'},
    margin={'l':200},
    showlegend=False
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Có thể trường hợp mà tổng reward cho thuật toán với giá trị regret thấp nhất sẽ không phải là cao nhất. Điều này là bởi thực tế rằng: dù cho thuật toán chọn ra được đúng quảng cáo thì nó cũng không đảm bảo rằng khách hàng sẽ click vào xem.

Như đã nói từ đầu, Lấy mẫu Thompson nhìn chung là lựa chọn tốt nhất, nhưng ta cũng xem thêm các thuật toán khác và thảo luận về cách thức và thời điểm tụi nó phát huy tác dụng. Ta lựa chọn phương pháp nào cho bài toán của mình sẽ tùy thuộc vào mỗi bái toán, thông tin ban đầu và kết quả muốn nhận được sau cùng.